In [2]:
#All the basic Imports Needed to Run the application
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, BatchNormalization
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os

In [ ]:
#Declaring Variables for Train and Test Data Directory
train_data_dir = './refined_dataset/train'
validation_data_dir = './refined_dataset/test'

#Data Augmentation to get all the Data in Different Mannner
train_datagen = ImageDataGenerator(rescale=1./255, rotation_range=30, width_shift_range=0.3, height_shift_range=0.3, horizontal_flip=False, fill_mode='nearest')
validation_datagen = ImageDataGenerator(rescale=1./255)

#Lets Make Generators for getting The Data and Look at Size of dataset
train_generator = train_datagen.flow_from_directory(train_data_dir, target_size=(img_rows, img_cols), batch_size=batch_size, class_mode='categorical')